In [1]:
# --- Mount Drive ---
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import cv2


# --- Helper Functions ---------------------------------------------------------

def cut_clip(input_path, start_time, duration, output_path):
    """Extract a video clip using ffmpeg."""
    cmd = [
        "ffmpeg", "-y",
        "-i", input_path,
        "-ss", start_time,
        "-t", duration,
        "-c", "copy",
        output_path
    ]
    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


def get_video_length(path):
    """Return video length in seconds."""
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    cap.release()
    return frames / fps


def seconds_to_hms(seconds):
    """Convert seconds -> HH:MM:SS."""
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"


# --- Paths -------------------------------------------------------------------

base_dir = "/content/drive/MyDrive/Harvard/LS100/videos"
output_root = "/content/drive/MyDrive/Harvard/LS100/extracted_clips"
os.makedirs(output_root, exist_ok=True)

participants = ["Aryeh", "Danny", "Max"]


# --- Process each participant ------------------------------------------------

for person in participants:
    print(f"\n=== Processing {person} ===")

    person_dir = os.path.join(base_dir, person, f"{person}FR")
    output_dir = os.path.join(output_root, person)
    os.makedirs(output_dir, exist_ok=True)

    # Identify videos matching:   {Name}_1_FrameReduced.MP4 and {Name}_2_FrameReduced.MP4
    video1 = os.path.join(person_dir, f"{person}_1_FrameReduced.MP4")
    video2 = os.path.join(person_dir, f"{person}_2_FrameReduced.MP4")

    # Safety check
    if not os.path.exists(video1) or not os.path.exists(video2):
        print(f"ERROR: Missing videos for {person}")
        print("Expected:")
        print(f"  {video1}")
        print(f"  {video2}")
        continue

    # --- 1. Warmup: 0:01–1:00 ---
    cut_clip(video1, "00:00:01", "00:00:59",
             os.path.join(output_dir, f"{person}_warmup.mp4"))

    # --- 2. Run 1: 4:01–5:00 ---
    cut_clip(video1, "00:04:01", "00:00:59",
             os.path.join(output_dir, f"{person}_run1.mp4"))

    # --- 3. Run 2: 13:01–14:00 ---
    cut_clip(video1, "00:13:01", "00:00:59",
             os.path.join(output_dir, f"{person}_run2.mp4"))

    # --- 4. Run 3: 8:04–9:03 of video2 ---
    cut_clip(video2, "00:08:04", "00:00:59",
             os.path.join(output_dir, f"{person}_run3.mp4"))

    # --- 5. Cooldown: last 1 minute of video2 ---
    v2_length = get_video_length(video2)
    start_sec = max(v2_length - 60, 0)
    cut_clip(
        video2,
        seconds_to_hms(start_sec),
        "00:01:00",
        os.path.join(output_dir, f"{person}_cooldown.mp4")
    )

    print(f"{person}: all 5 clips extracted.")

print("\n=== DONE processing all participants ===")


Mounted at /content/drive

=== Processing Aryeh ===
Aryeh: all 5 clips extracted.

=== Processing Danny ===
Danny: all 5 clips extracted.

=== Processing Max ===
Max: all 5 clips extracted.

=== DONE processing all participants ===
